# OpenAltimetry ICESat-2: Elevation Workflows


### Overview
<blockquote>This notebook pulls a subset of ATL06, ATL07, ATL08, ATL10, ATL12 or ATL13 elevation data from the ICESat 2 mission via the OpenAltimetry API. This API enables users to select a level 3a product, spatial region of interest and a track to get elevation data without the need to deal with large HDF files.</blockquote>



### Instructions
<blockquote>Get an API end point from any elevation plot in OpenAltimetry.org and paste it in the **OA_API_URL** below to start visualizing your data!

Limitations - Date range is limited to 1 year. If no date range is specified, last 1 year's worth of data is returned. Spatial area is limited to 5x5 deg.</blockquote>


Note that this notebook already has the Python libraries that you might need to do data analysis such as geopandas, plotly, fiona, shapely and more!

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot

# We populate a list with the elevation data using the OpenAltimetry API

print('Requesting data from OA')

OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/level3a?product=atl08&startDate=2019-07-01&endDate=2019-12-30&minx=-102.27786&miny=35.49232&maxx=-101.91179&maxy=36.30839&trackId=752&client=jupyter&outputFormat=json'

dataFrame = []
oa_plots = []

r = requests.get(OA_API_URL)
elevation_data = r.json()

product = elevation_data["product"]

colorChart = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']

#print (colorChart)
print('Product: ' + product)
print('TrackId: ' + elevation_data["trackId"])

colorIndex = -1

for data_date in elevation_data['data']:
    date = data_date["date"]
    scForward = data_date["scForward"]
    data_beams = data_date["beams"]
    
    print('date: ' + date + '\tscForward: ' + str(scForward) + '\tTotal data beams: ' + str(len(data_beams)))

    colorIndex += 1
    colorIndex = colorIndex % 6
    
    for data_beam in data_beams:
        beam = data_beam["beam"]
        isStrongBeam = data_beam["isStrongBeam"]        
        data_name = 'lat_lon_elev'
        if product == 'ATL08':
            data_name = 'lat_lon_elev_canopy'
            
        series = []            
        for p in data_beam[data_name]:
            series.append({
                'lat': p[0],
                'lon': p[1],
                'h': p[2]
            })

        strong_beam = ''
        if (isStrongBeam):
            strong_beam = 'strong'
        else:
            strong_beam = 'weak'
        
        print ('\tbeam: ' + beam + ',\t' + strong_beam + ', \tseries size: ' + str(len(series)))
        
        if (len(series) > 0):
            df = pd.DataFrame.from_dict(series)
            df.name = date + '_' + beam
            df.color = colorChart[colorIndex]
            
            #print ('\t\tColor by date: ' + df.color)
            dataFrame.append(df) 
    
print('dataFrame size: ' + str(len(dataFrame)))

for df in dataFrame:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=1,
                                        color=df.color
                                    ),
                                    line=dict(
                                        color=df.color,
                                        width=1
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)
    
